# Fantasy Hockey Draft

This notebook will simulate a fantasy hockey draft using Markowitz optimization strategies, and also supports the possiblity of a human player. 

In [ ]:
import numpy as np
import pandas as pd
import cvxpy as cp
import importlib
import sys

sys.path.insert(1, '../')
import scripts.overwatch_optim as ow

# need to keep reloading for development work because 
# I apparently like Jupyter too much 
importlib.reload(ow)

## Importing Data Created in `Data Wrangling.ipynb`

In [ ]:
player_data = pd.read_csv("../../with_date.csv")
player_data= player_data[(player_data.date < '2019-08-15') & (player_data.date > '2019-02-14')]
# aggregate all scores into an array for each player
player_data = player_data.dropna(subset=['name'])
scores = player_data[['name', 'score',]].groupby('name').agg(lambda x: list(x)).reset_index()
#all_points = pd.DataFrame(scores.score.tolist(), columns = scores.name)
#
# pd.DataFrame(np.transpose(scores.score), columns = scores.name)
a = pd.DataFrame(scores.score.to_list()).T
a.columns = scores.name
a = a.fillna(0)
all_points=a.copy()
idx = list(all_points.mean().sort_values(ascending=False).index)
    

## Finding Indexes
As we're only interested in aggregate inddexes of average player returns, we need to find each players index according to the aggregation, as well s the indexes of the players in each position done below.

In [ ]:
# Finding index in an aggregate score for each position
pointies = list(all_points.mean().index)

offence = ow.position_indexes(pointies,all_points,player_data,idx, "offense")
tank = ow.position_indexes(pointies,all_points,player_data,idx, "tank")
support = ow.position_indexes(pointies,all_points, player_data,idx,"support")



## Running the Draft

Here we run the actual draft. Noe that `order` defines the order of the draft and automatically reverses after each round. Also note that the teams will be returned in the order that tay appear in the `functions` list in the final result. 

In [ ]:
importlib.reload(ow)

functions = [ow.optim_player, ow.optim_player,
             ow.optim_player, ow.optim_player]
#              ow.optim_player, 
#              ow.optim_player, 
#             ow.optim_player, 
#             ow.optim_player,
#             ow.optim_player]

greedy_selections = {}
greedy_selections['goalie'] = []
greedy_selections['defence'] = []
greedy_selections['center'] = []
greedy_selections['right_winger'] = []
greedy_selections['left_winger'] = []


order = [0,1,2,3]

assert len(order) == len(functions)

args = dict(scores = all_points, 
            gammaa = [1, .001, .5, 0], 
            greedy_selections = greedy_selections,
            df = player_data,
            offence = offence,
            tank = tank,
            support = support,
            selection = ['max', 'optim', 'rms', 'max', 'optim', 'optim', 'optim', 'optim'],
            sub_gamma = [None, .0001, None, None, .2, .8, .3, .4])
taken = []
all_players, teams = ow.draft(functions, order, pause = False, team_size=6, **args)





In [ ]:
ohboy = player_data[player_data.name.isin(list(all_points.iloc[:,teams[order[1]]]))]
print(ohboy.name.unique())
print(ohboy['score'].sum())
print(ohboy['score'].mean())
ohboy['score'].hist()

In [ ]:
ohboy.drop_duplicates(subset='name')